In [11]:
import glob, json, os
import pandas as pd
import yaml

jsonFileList = glob.glob('reports-data/*.json')
output_dir = '../_datasets'
metadata_file_path = os.path.join('metadata.yaml')

df = pd.DataFrame()
for file in jsonFileList:
	with open(file, "r", encoding='utf-8') as f:
		data = json.load(f)
		each_df = pd.DataFrame(data)
		each_df['pre_existing_conditions'] = each_df['pre_existing_conditions'].str.replace('\r\n', '\n').str.replace('\n', '')
		df = pd.concat([df, each_df])

In [ ]:
start_with_id_columns = df.columns.delete(15).to_list()
start_with_id_columns.insert(0, "id")
df = df.reindex(columns=start_with_id_columns)
df = df.reset_index(drop=True)

In [ ]:
duplicated_df = df[df["id"].duplicated()].iloc[:, [0, 2, 3]]
if len(duplicated_df) > 0:
	print("[Error] idが重複したデータがあります。修正してください。")
	print(duplicated_df)